In [1]:
import pandas as pd 
import requests

C:\Users\Estudiante\AppData\Local\Temp\ipykernel_3244\1337425083.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Set up the API endpoint and API key
base_url = "https://thecocktaildb.com/api/json/v2/"
api_key = "1"

In [3]:
# Endpoint to retrieve the list of ingredients
endpoint = f"{base_url}{api_key}/list.php?i=list"

# Send GET request
response = requests.get(endpoint)

# Get the JSON response
data = response.json()

# Process the data
if data['drinks']:
    ingredients = [ingredient['strIngredient1'] for ingredient in data['drinks']]
    # Access the list of ingredients, e.g., ingredients
else:
    print("No ingredients found.")

# Print the list of ingredients
for ingredient in ingredients:
    print(ingredient)

Light rum
Applejack
Gin
Dark rum
Sweet Vermouth
Strawberry schnapps
Scotch
Apricot brandy
Triple sec
Southern Comfort
Orange bitters
Brandy
Lemon vodka
Blended whiskey
Dry Vermouth
Amaretto
Tea
Champagne
Coffee liqueur
Bourbon
Tequila
Vodka
Añejo rum
Bitters
Sugar
Kahlua
demerara Sugar
Dubonnet Rouge
Watermelon
Lime juice
Irish whiskey
Apple brandy
Carbonated water
Cherry brandy
Creme de Cacao
Grenadine
Port
Coffee brandy
Red wine
Rum
Grapefruit juice
Ricard
Sherry
Cognac
Sloe gin
Apple juice
Pineapple juice
Lemon juice
Sugar syrup
Milk
Strawberries
Chocolate syrup
Yoghurt
Mango
Ginger
Lime
Cantaloupe
Berries
Grapes
Kiwi
Tomato juice
Cocoa powder
Chocolate
Heavy cream
Galliano
Peach Vodka
Ouzo
Coffee
Spiced rum
Water
Espresso
Angelica root
Orange
Cranberries
Johnnie Walker
Apple cider
Everclear
Cranberry juice
Egg yolk
Egg
Grape juice
Peach nectar
Lemon
Firewater
Lemonade
Lager
Whiskey
Absolut Citron
Pisco
Irish cream
Ale
Chocolate liqueur
Midori melon liqueur
Sambuca
Cider
Sprite
7-Up

In [4]:
# Example request: Search cocktails by each ingredient
ingredients
cocktail_ids = []
cocktails = []  # Define an empty list to store the retrieved cocktails

for ingredient in ingredients:
    endpoint = f"{base_url}{api_key}/filter.php?i={ingredient}"
    
    # Send GET request
    response = requests.get(endpoint)
    
    # Get the JSON response
    data = response.json()
    
    # Process the data
    if data['drinks']:
        for cocktail in data['drinks']:
            cocktail_ids.append(cocktail['idDrink'])
    else:
        print(f"No cocktails found with ingredient: {ingredient}")

for cocktail_id in cocktail_ids:
    endpoint = f"{base_url}{api_key}/lookup.php?i={cocktail_id}"

    # Send GET request
    response = requests.get(endpoint)

    # Get the JSON response
    data = response.json()

    # Process the data as needed
    if data['drinks']:
        cocktail = data['drinks'][0]
        cocktails.append(cocktail)
    else:
        print(f"Cocktail with ID {cocktail_id} not found.")

In [43]:
df = pd.json_normalize(cocktails)

In [46]:
df.columns

Index(['idDrink', 'strDrink', 'strDrinkAlternate', 'strTags', 'strVideo',
       'strCategory', 'strIBA', 'strAlcoholic', 'strGlass', 'strInstructions',
       'strInstructionsES', 'strInstructionsDE', 'strInstructionsFR',
       'strInstructionsIT', 'strInstructionsZH-HANS', 'strInstructionsZH-HANT',
       'strDrinkThumb', 'strIngredient1', 'strIngredient2', 'strIngredient3',
       'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7',
       'strIngredient8', 'strIngredient9', 'strIngredient10',
       'strIngredient11', 'strIngredient12', 'strIngredient13',
       'strIngredient14', 'strIngredient15', 'strMeasure1', 'strMeasure2',
       'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
       'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10',
       'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14',
       'strMeasure15', 'strImageSource', 'strImageAttribution',
       'strCreativeCommonsConfirmed', 'dateModified'],
      dtype='o

In [45]:
df.isna().sum()

idDrink                          0
strDrink                         0
strDrinkAlternate              266
strTags                        193
strVideo                       253
strCategory                      0
strIBA                         223
strAlcoholic                     0
strGlass                         0
strInstructions                  0
strInstructionsES              223
strInstructionsDE               35
strInstructionsFR              266
strInstructionsIT                6
strInstructionsZH-HANS         266
strInstructionsZH-HANT         266
strDrinkThumb                    0
strIngredient1                   0
strIngredient2                   0
strIngredient3                  20
strIngredient4                  79
strIngredient5                 142
strIngredient6                 195
strIngredient7                 238
strIngredient8                 251
strIngredient9                 262
strIngredient10                266
strIngredient11                266
strIngredient12     

In [48]:
df.duplicated

23